In [ ]:
ranks = ['7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
suits =  ['Hearts', 'Diamonds', 'Clubs', 'Spades']

In [ ]:
points = [0, 0, 2, 1, 3, 0, 0, 1.1]

In [ ]:
deck_1 = []
for i in ranks:
    for j in suits:
        deck_1.append('{} of {}'.format(i, j))

In [ ]:
deck_1

['7 of Hearts',
 '7 of Diamonds',
 '7 of Clubs',
 '7 of Spades',
 '8 of Hearts',
 '8 of Diamonds',
 '8 of Clubs',
 '8 of Spades',
 '9 of Hearts',
 '9 of Diamonds',
 '9 of Clubs',
 '9 of Spades',
 '10 of Hearts',
 '10 of Diamonds',
 '10 of Clubs',
 '10 of Spades',
 'Jack of Hearts',
 'Jack of Diamonds',
 'Jack of Clubs',
 'Jack of Spades',
 'Queen of Hearts',
 'Queen of Diamonds',
 'Queen of Clubs',
 'Queen of Spades',
 'King of Hearts',
 'King of Diamonds',
 'King of Clubs',
 'King of Spades',
 'Ace of Hearts',
 'Ace of Diamonds',
 'Ace of Clubs',
 'Ace of Spades']

In [5]:
import numpy as np

In [6]:
points = [0]*8+[2]*4+[1]*4+[3]*4+[0]*8+[1.1]*4

In [7]:
points

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.1,
 1.1,
 1.1,
 1.1]

In [8]:
suits = []
for a in deck_1:
    suits.append(a.split(' ')[-1])

In [9]:
suits

['Hearts',
 'Diamonds',
 'Clubs',
 'Spades',
 'Hearts',
 'Diamonds',
 'Clubs',
 'Spades',
 'Hearts',
 'Diamonds',
 'Clubs',
 'Spades',
 'Hearts',
 'Diamonds',
 'Clubs',
 'Spades',
 'Hearts',
 'Diamonds',
 'Clubs',
 'Spades',
 'Hearts',
 'Diamonds',
 'Clubs',
 'Spades',
 'Hearts',
 'Diamonds',
 'Clubs',
 'Spades',
 'Hearts',
 'Diamonds',
 'Clubs',
 'Spades']

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame({'Name': deck_1, 'Suit': suits, 'Points': points})

In [12]:
df = df.sample(frac=1).reset_index(drop=True)

In [13]:
df.to_csv('output.csv')

In [14]:
df

,Name,Suit,Points
0,King of Diamonds,Diamonds,0.0
1,Ace of Spades,Spades,1.1
2,Ace of Diamonds,Diamonds,1.1
3,King of Spades,Spades,0.0
4,Queen of Clubs,Clubs,0.0
5,10 of Hearts,Hearts,1.0
6,Queen of Spades,Spades,0.0
7,Jack of Spades,Spades,3.0
8,9 of Spades,Spades,2.0
9,Jack of Clubs,Clubs,3.0


In [74]:
class Player():
    def __init__(self, cards, name):
        self.cards = cards
        self.colour_made = False
        self.points_made = 0
        self.name = name
        self.hands_won = []

class Robot():
    def __init__(self, cards, name):
        self.cards = cards
        self.colour_made = False
        self.points_made = 0
        self.name = name
        self.hands_won = []
class Game():
    def __init__(self):
        self.color = None
        self.last_hand_winner = None
        self.digged = False


In [16]:
player = Player(df[0:8], 'Adi')
robot_1 = Robot(df[8:16], 'Robot 1')
robot_2 = Robot(df[16:24], 'Robot 2')
robot_3 = Robot(df[24:32], 'Robot 3')

In [17]:
class Bid():
    def __init__(self, b1, b2):
        self.how_much_b1 = 17
        self.how_much_b2 = 17
        self.b1_bid = input(f"Do you want to bid {b1.name}? Y/N: ").lower()
        
        if self.b1_bid == "n":
            print(f"{b2.name} wins the bid as {b1.name} passes at {self.how_much_b2}")
            self.winner = b2
        elif self.b1_bid == "y":
            self.b2_bid = input(f"Do you want to bid {b2.name}? Y/N: ").lower()

            if self.b2_bid == "n":
                print(f"{b1.name} wins the bid as {b2.name} passes at {self.how_much_b1}")
                self.winner = b1
            else:
                while self.b1_bid == "y" and self.b2_bid == "y":
                    self.how_much_b1 = int(input(f"How much would you like to bid {b1.name}? (minimum bid is 17): "))
                    self.how_much_b2 = int(input(f"How much would you like to bid {b2.name}? (minimum bid is 17): "))

                    # Ensure minimum bid
                    while self.how_much_b1 < 17 or self.how_much_b2 < 17:
                        print("Invalid bid! The minimum bid is 17.")
                        self.how_much_b1 = int(input(f"How much would you like to bid {b1.name}? (minimum bid is 17): "))
                        self.how_much_b2 = int(input(f"How much would you like to bid {b2.name}? (minimum bid is 17): "))

                    # Determine winner of the bid
                    if self.how_much_b1 < self.how_much_b2:
                        self.b1_bid = input(f"{b2.name} is winning the bid. Do you want to continue, {b1.name}? Y/N: ").lower()
                        if self.b1_bid == "n":
                            print(f"{b2.name} wins the bid as {b1.name} passes at {self.how_much_b2}")
                            self.winner = b2
                    elif self.how_much_b1 > self.how_much_b2:
                        self.b2_bid = input(f"{b1.name} is winning the bid. Do you want to continue, {b2.name}? Y/N: ").lower()
                        if self.b2_bid == "n":
                            print(f"{b1.name} wins the bid as {b2.name} passes at {self.how_much_b1}")
                            self.winner = b1


In [18]:
df


,Name,Suit,Points
0,King of Diamonds,Diamonds,0.0
1,Ace of Spades,Spades,1.1
2,Ace of Diamonds,Diamonds,1.1
3,King of Spades,Spades,0.0
4,Queen of Clubs,Clubs,0.0
5,10 of Hearts,Hearts,1.0
6,Queen of Spades,Spades,0.0
7,Jack of Spades,Spades,3.0
8,9 of Spades,Spades,2.0
9,Jack of Clubs,Clubs,3.0


In [19]:
print(df)

                 Name      Suit  Points
0    King of Diamonds  Diamonds     0.0
1       Ace of Spades    Spades     1.1
2     Ace of Diamonds  Diamonds     1.1
3      King of Spades    Spades     0.0
4      Queen of Clubs     Clubs     0.0
5        10 of Hearts    Hearts     1.0
6     Queen of Spades    Spades     0.0
7      Jack of Spades    Spades     3.0
8         9 of Spades    Spades     2.0
9       Jack of Clubs     Clubs     3.0
10     King of Hearts    Hearts     0.0
11      8 of Diamonds  Diamonds     0.0
12    Queen of Hearts    Hearts     0.0
13        8 of Hearts    Hearts     0.0
14   Jack of Diamonds  Diamonds     3.0
15      Ace of Hearts    Hearts     1.1
16         7 of Clubs     Clubs     0.0
17      7 of Diamonds  Diamonds     0.0
18       10 of Spades    Spades     1.0
19      9 of Diamonds  Diamonds     2.0
20  Queen of Diamonds  Diamonds     0.0
21     10 of Diamonds  Diamonds     1.0
22         9 of Clubs     Clubs     2.0
23         8 of Clubs     Clubs     0.0


In [20]:
if robot_1.cards["Points"].sum() < 8:
    print("hi")

In [21]:
player.cards 


,Name,Suit,Points
0,King of Diamonds,Diamonds,0.0
1,Ace of Spades,Spades,1.1
2,Ace of Diamonds,Diamonds,1.1
3,King of Spades,Spades,0.0
4,Queen of Clubs,Clubs,0.0
5,10 of Hearts,Hearts,1.0
6,Queen of Spades,Spades,0.0
7,Jack of Spades,Spades,3.0


In [22]:
robot_1.cards


,Name,Suit,Points
8,9 of Spades,Spades,2.0
9,Jack of Clubs,Clubs,3.0
10,King of Hearts,Hearts,0.0
11,8 of Diamonds,Diamonds,0.0
12,Queen of Hearts,Hearts,0.0
13,8 of Hearts,Hearts,0.0
14,Jack of Diamonds,Diamonds,3.0
15,Ace of Hearts,Hearts,1.1


In [23]:
robot_2.cards


,Name,Suit,Points
16,7 of Clubs,Clubs,0.0
17,7 of Diamonds,Diamonds,0.0
18,10 of Spades,Spades,1.0
19,9 of Diamonds,Diamonds,2.0
20,Queen of Diamonds,Diamonds,0.0
21,10 of Diamonds,Diamonds,1.0
22,9 of Clubs,Clubs,2.0
23,8 of Clubs,Clubs,0.0


In [24]:
robot_3.cards

,Name,Suit,Points
24,Ace of Clubs,Clubs,1.1
25,9 of Hearts,Hearts,2.0
26,8 of Spades,Spades,0.0
27,King of Clubs,Clubs,0.0
28,7 of Spades,Spades,0.0
29,7 of Hearts,Hearts,0.0
30,10 of Clubs,Clubs,1.0
31,Jack of Hearts,Hearts,3.0


In [25]:
deck_1

['7 of Hearts',
 '7 of Diamonds',
 '7 of Clubs',
 '7 of Spades',
 '8 of Hearts',
 '8 of Diamonds',
 '8 of Clubs',
 '8 of Spades',
 '9 of Hearts',
 '9 of Diamonds',
 '9 of Clubs',
 '9 of Spades',
 '10 of Hearts',
 '10 of Diamonds',
 '10 of Clubs',
 '10 of Spades',
 'Jack of Hearts',
 'Jack of Diamonds',
 'Jack of Clubs',
 'Jack of Spades',
 'Queen of Hearts',
 'Queen of Diamonds',
 'Queen of Clubs',
 'Queen of Spades',
 'King of Hearts',
 'King of Diamonds',
 'King of Clubs',
 'King of Spades',
 'Ace of Hearts',
 'Ace of Diamonds',
 'Ace of Clubs',
 'Ace of Spades']

In [26]:
df

,Name,Suit,Points
0,King of Diamonds,Diamonds,0.0
1,Ace of Spades,Spades,1.1
2,Ace of Diamonds,Diamonds,1.1
3,King of Spades,Spades,0.0
4,Queen of Clubs,Clubs,0.0
5,10 of Hearts,Hearts,1.0
6,Queen of Spades,Spades,0.0
7,Jack of Spades,Spades,3.0
8,9 of Spades,Spades,2.0
9,Jack of Clubs,Clubs,3.0


In [27]:
# player.cards.drop(player.cards[player.cards['Name'] == '7 of Clubs'].index)

In [28]:
game = Game()

In [29]:
current_hand = pd.DataFrame(columns = df.columns)

In [30]:
def shuffle_deck():
    global df
    df = df.sample(frac = 1, ignore_index = True)

In [31]:
def distribute_cards():
    global df
    global player, robot_1, robot_2, robot_3
    player = Player(df[0:8], input("What is your name? "))
    robot_1 = Robot(df[8:16], 'Robot 1')
    robot_2 = Robot(df[16:24], 'Robot 2')
    robot_3 = Robot(df[24:32], 'Robot 3')

In [32]:
def play_card(name, player):
    global current_hand
    current_hand = pd.concat([current_hand, df[df['Name'] == name]], ignore_index = True)
    # pd.concat([new_data, data], ignore_index=True)
    player.cards = player.cards.drop(player.cards[player.cards['Name'] == name].index)

In [45]:
def setup():
    shuffle_deck() # Shuffling all the cards
    distribute_cards() # Distributing all the cards
    print("Nice to meet you {}".format(player.name))
    print("Welcome to our newly made game of 29. We hope you enjoy it")
    print(player.cards[:4]) # Showing the player their first 4 cards
    global game
    
    # BIDDING
    player_bid = int(input('How much do you want to bid?: '))
    
    # robot_1
    r1_bid = 0
    if len(robot_1.cards[:4]['Suit'].mode() == 1):
        r1_mode_suit = robot_1.cards[:4]['Suit'].mode().iloc[0]
        r1_color = robot_1.cards[:4]['Suit'].value_counts()[r1_mode_suit]
        r1_points = robot_1.cards[:4][robot_1.cards[:4]['Suit'] == r1_mode_suit]['Points'].sum()
        
        if r1_points == 3 or r1_points == 3.1:
            if r1_color == 2: r1_bid = 18
            if r1_color == 3: r1_bid = 19
        if r1_points == 4 or r1_points == 4.1:
            if r1_color == 2: r1_bid = 18
            if r1_color == 3: r1_bid = 19
        if r1_points == 5 or r1_points == 5.1:
            if r1_color == 2: r1_bid = 19
            if r1_color == 3: r1_bid = 20
        if r1_points == 6 or r1_points == 6.1:
            if r1_color == 3: r1_bid = 21
        if r1_points == 6 or r1_points == 6.1 or r1_points == 7.1:
            if r1_color == 4: r1_bid = 22
    
    # robot_2
    r2_bid = 0
    if len(robot_2.cards[:4]['Suit'].mode() == 2):
        r2_mode_suit = robot_2.cards[:4]['Suit'].mode().iloc[0]
        r2_color = robot_2.cards[:4]['Suit'].value_counts()[r2_mode_suit]
        r2_points = robot_2.cards[:4][robot_2.cards[:4]['Suit'] == r2_mode_suit]['Points'].sum()
        
        if r2_points == 3 or r2_points == 3.1:
            if r2_color == 2: r2_bid = 18
            if r2_color == 3: r2_bid = 19
        if r2_points == 4 or r2_points == 4.1:
            if r2_color == 2: r2_bid = 18
            if r2_color == 3: r2_bid = 19
        if r2_points == 5 or r2_points == 5.1:
            if r2_color == 2: r2_bid = 19
            if r2_color == 3: r2_bid = 20
        if r2_points == 6 or r2_points == 6.1:
            if r2_color == 3: r2_bid = 21
        if r2_points == 6 or r2_points == 6.1 or r2_points == 7.1:
            if r2_color == 4: r2_bid = 22
    
    # robot_3
    r3_bid = 0
    if len(robot_3.cards[:4]['Suit'].mode() == 3):
        r3_mode_suit = robot_3.cards[:4]['Suit'].mode().iloc[0]
        r3_color = robot_3.cards[:4]['Suit'].value_counts()[r3_mode_suit]
        r3_points = robot_3.cards[:4][robot_3.cards[:4]['Suit'] == r3_mode_suit]['Points'].sum()
        
        if r3_points == 3 or r3_points == 3.1:
            if r3_color == 2: r3_bid = 18
            if r3_color == 3: r3_bid = 19
        if r3_points == 4 or r3_points == 4.1:
            if r3_color == 2: r3_bid = 18
            if r3_color == 3: r3_bid = 19
        if r3_points == 5 or r3_points == 5.1:
            if r3_color == 2: r3_bid = 19
            if r3_color == 3: r3_bid = 20
        if r3_points == 6 or r3_points == 6.1:
            if r3_color == 3: r3_bid = 22
        if r3_points == 6 or r3_points == 6.1 or r3_points == 7.1:
            if r3_color == 4: r3_bid = 22
    bidding = [player_bid, r1_bid, r2_bid, r3_bid]
    max_bid = max(bidding)
    max_bid_index = bidding.index(max_bid)
    if max_bid_index == 0: winner = player.name
    if max_bid_index == 1: winner = robot_1.name
    if max_bid_index == 2: winner = robot_2.name
    if max_bid_index == 3: winner = robot_3.name
    print('{} won the bet and will make the color on {} points.'.format(winner, max_bid))
    if winner == player.name:
        player.colour_made = True
        game.color = input(f"What is the colour {player.name}?")
    if winner == robot_1.name:
        game.color = r1_mode_suit
    if winner == robot_2.name:
        game.color = r2_mode_suit
    if winner == robot_3.name:
        game.color = r3_mode_suit
    game.color = game.color.capitalize()

In [46]:
def flow_chart(r):
    global current_hand, game
    global df
    r.suits = list(r.cards["Suit"])
    current_suit = current_hand.iloc[0]['Suit']
    
    if current_suit in r.suits:
        how_many = r.suits.count(current_suit)
        
        if how_many == 1:
            card_to_play = r.cards[r.cards['Suit'] == current_suit]['Name']
            card_to_play = card_to_play.iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)

        if how_many > 1:
            
            if len(r.cards[(r.cards['Suit'] == current_suit) & (r.cards['Points'] == 3)]) == 0:
                cs = r.cards[r.cards['Suit'] == current_suit]
                card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
                card_to_play = card_to_play.iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)
            
            else: 
                card_to_play = r.cards[(r.cards['Suit'] == current_suit) & (r.cards['Points'] == 3)]['Name']
                card_to_play = card_to_play.iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)
            
    if not game.digged and current_suit not in r.suits:
        print(f'{r.name} has revealed the trump card. It is {game.color}')
        if len(r.cards[(r.cards['Suit'] == game.color)]) > 0:
            cs = r.cards[r.cards['Suit'] == game.color]
            card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
            card_to_play = card_to_play.iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)
        else: 
            card_to_play = r.cards.sample(n = 1)['Name'].iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)
        game.digged = True
    elif game.digged and current_suit not in r.suits:
        if len(r.cards[(r.cards['Suit'] == game.color)]) > 0:
            cs = r.cards[r.cards['Suit'] == game.color]
            card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
            card_to_play = card_to_play.iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)
        else: 
            card_to_play = r.cards.sample(n = 1)['Name'].iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)

In [80]:
def play_game():
    import warnings

    warnings.filterwarnings(
        "ignore",
        category=FutureWarning, 
        message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated"
    )

    #SETUP THE GAME
    setup()
    #DO THE GLOBALS
    global player, robot_1, robot_2, robot_3, game, current_hand, last_hand
    #print(player.cards)
    #PLAYE ALL THE 8 HANDS
    for i in range(0, 8):
        print(f"Here are your cards {player.name}: \n {player.cards}")
        card = input(f"What card do you wanna play {player.name}")
        card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
        play_card(card, player)
        flow_chart(robot_1)
        flow_chart(robot_2)
        flow_chart(robot_3)
        if game.digged and game.color in current_hand:
            winner_index = current_hand[current_hand['Suit'] == game.color]['Points'].max()
        elif not game.digged:
            winner_index = current_hand[current_hand['Suit'] == current_hand.iloc[0]['Suit']]['Points'].max()
        
            
            
        last_hand = current_hand
        current_hand = pd.DataFrame(data = {}, columns = df.columns)

In [49]:
 play_game()

What is your name? a
Nice to meet you a
Welcome to our newly made game of 29. We hope you enjoy it
             Name      Suit  Points
0   7 of Diamonds  Diamonds     0.0
1  Queen of Clubs     Clubs     0.0
2   Ace of Hearts    Hearts     1.1
3  King of Hearts    Hearts     0.0
How much do you want to bid?: 0
Robot 3 won the bet and will make the color on 19 points.
Here are your cards a: 
                Name      Suit  Points
0     7 of Diamonds  Diamonds     0.0
1    Queen of Clubs     Clubs     0.0
2     Ace of Hearts    Hearts     1.1
3    King of Hearts    Hearts     0.0
4  King of Diamonds  Diamonds     0.0
5       9 of Spades    Spades     2.0
6       7 of Spades    Spades     0.0
7     9 of Diamonds  Diamonds     2.0
What card do you wanna play a9 of Spades
Robot 1 has revealed the trump card. It is Clubs
Robot 1 plays King of Clubs
Robot 2 plays 8 of Spades
Robot 3 plays Jack of Spades
Here are your cards a: 
                Name      Suit  Points
0     7 of Diamonds  Diamond

KeyboardInterrupt: Interrupted by user

In [ ]:
flow_chart(robot_1)

In [ ]:
flow_chart(robot_2)

In [ ]:
flow_chart(robot_3)

In [ ]:
current_hand

In [ ]:
player.cards

In [48]:
robot_1.cards

,Name,Suit,Points
8,8 of Hearts,Hearts,0.0
9,8 of Diamonds,Diamonds,0.0
10,Queen of Diamonds,Diamonds,0.0
11,King of Diamonds,Diamonds,0.0
12,Queen of Hearts,Hearts,0.0
13,7 of Hearts,Hearts,0.0
14,Queen of Clubs,Clubs,0.0
15,7 of Diamonds,Diamonds,0.0


In [44]:
robot_2.cards

,Name,Suit,Points
16,9 of Diamonds,Diamonds,2.0
17,10 of Diamonds,Diamonds,1.0
18,Jack of Diamonds,Diamonds,3.0
19,7 of Clubs,Clubs,0.0
20,Jack of Hearts,Hearts,3.0
21,8 of Clubs,Clubs,0.0
22,Ace of Hearts,Hearts,1.1
23,Ace of Clubs,Clubs,1.1


In [ ]:
current_hand

In [ ]:
current_hand = pd.DataFrame(data = {}, columns = df.columns)

In [ ]:
current_hand

In [ ]:
last_hand

In [ ]:
import warnings

warnings.filterwarnings(
    "ignore",
    category=FutureWarning, 
    message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated"
)


In [50]:
current_hand

,Name,Suit,Points


In [51]:
last_hand

,Name,Suit,Points
0,9 of Spades,Spades,2.0
1,King of Clubs,Clubs,0.0
2,8 of Spades,Spades,0.0
3,Jack of Spades,Spades,3.0


In [54]:
last_hand[last_hand['Suit'] == game.color].index

Index([1], dtype='int64')

In [61]:
if game.color in list(last_hand['Suit']): print('True')
else: print('False')

True


In [71]:
s = None

In [73]:
s

In [75]:
last_hand

,Name,Suit,Points
0,9 of Spades,Spades,2.0
1,King of Clubs,Clubs,0.0
2,8 of Spades,Spades,0.0
3,Jack of Spades,Spades,3.0


In [77]:
last_hand.iloc[0]['Suit']

'Spades'